In [41]:
import pandas as pd
import numpy as np
import seaborn as sns
color = sns.color_palette('PuBuGn_d')
sns.set_style('darkgrid')
from scipy import stats
import matplotlib.pyplot as plt  # Matlab-style plotting

In [42]:
data=pd.read_csv("train.csv")

In [43]:
data.head(1)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


In [44]:
train=data.drop(["Id","SalePrice"],axis=1)

In [45]:
target = data["SalePrice"]

In [46]:
train['LotFrontage']=train['LotFrontage'].fillna(train['LotFrontage'].mean())

In [47]:
train['Alley']=train['Alley'].fillna('None')

In [48]:
train['MasVnrType']=train['MasVnrType'].fillna('None')

In [49]:
train['MasVnrArea']=train['MasVnrArea'].fillna(0)

In [50]:
for col in ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']:
    train[col]=train[col].fillna('None')

In [51]:
train['Electrical']=train['Electrical'].fillna('SBrkr')

In [52]:
train['FireplaceQu']=train['FireplaceQu'].fillna('None')

In [53]:
for col in ['GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond']:
    train[col]=train[col].fillna('None')

In [54]:
train['PoolQC']=train['PoolQC'].fillna('None')

In [55]:
train['Fence']=train['Fence'].fillna('None')

In [56]:
train['MiscFeature']=train['MiscFeature'].fillna('None')

In [57]:
for x in train['MSSubClass'].values:
    train.loc[train['MSSubClass']==x,'MSSubClass']='sub'+str(x)

train['MSSubClass'].astype('O')

0        sub60
1        sub20
2        sub60
3        sub70
4        sub60
5        sub50
6        sub20
7        sub60
8        sub50
9       sub190
10       sub20
11       sub60
12       sub20
13       sub20
14       sub20
15       sub45
16       sub20
17       sub90
18       sub20
19       sub20
20       sub60
21       sub45
22       sub20
23      sub120
24       sub20
25       sub20
26       sub20
27       sub20
28       sub20
29       sub30
         ...  
1430     sub60
1431    sub120
1432     sub30
1433     sub60
1434     sub20
1435     sub20
1436     sub20
1437     sub20
1438     sub20
1439     sub60
1440     sub70
1441    sub120
1442     sub60
1443     sub30
1444     sub20
1445     sub85
1446     sub20
1447     sub60
1448     sub50
1449    sub180
1450     sub90
1451     sub20
1452    sub180
1453     sub20
1454     sub20
1455     sub60
1456     sub20
1457     sub70
1458     sub20
1459     sub20
Name: MSSubClass, Length: 1460, dtype: object

In [58]:

train.loc[train['GarageYrBlt']=='None','GarageYrBlt']=0
train['GarageYrBlt'].astype('int64')

0       2003
1       1976
2       2001
3       1998
4       2000
5       1993
6       2004
7       1973
8       1931
9       1939
10      1965
11      2005
12      1962
13      2006
14      1960
15      1991
16      1970
17      1967
18      2004
19      1958
20      2005
21      1930
22      2002
23      1976
24      1968
25      2007
26      2005
27      2008
28      1957
29      1920
        ... 
1430    2005
1431    1976
1432    1928
1433    2000
1434    1977
1435    1962
1436    1974
1437    2008
1438    1957
1439    1979
1440    1993
1441    2004
1442    2008
1443    1916
1444    2004
1445    1990
1446    1962
1447    1995
1448    1950
1449       0
1450       0
1451    2008
1452    2005
1453       0
1454    2004
1455    1999
1456    1978
1457    1941
1458    1950
1459    1965
Name: GarageYrBlt, Length: 1460, dtype: int64

In [59]:
#fill missing values in columns which is data type of float64 and int64 
object_value={}
for col in train.columns:
    if train[col].dtype=='O':
        object_value[col]=dict(enumerate(train[col].unique()))
                               
object_value
object_value.pop('GarageYrBlt')
object_value

{'Alley': {0: 'None', 1: 'Grvl', 2: 'Pave'},
 'BldgType': {0: '1Fam', 1: '2fmCon', 2: 'Duplex', 3: 'TwnhsE', 4: 'Twnhs'},
 'BsmtCond': {0: 'TA', 1: 'Gd', 2: 'None', 3: 'Fa', 4: 'Po'},
 'BsmtExposure': {0: 'No', 1: 'Gd', 2: 'Mn', 3: 'Av', 4: 'None'},
 'BsmtFinType1': {0: 'GLQ',
  1: 'ALQ',
  2: 'Unf',
  3: 'Rec',
  4: 'BLQ',
  5: 'None',
  6: 'LwQ'},
 'BsmtFinType2': {0: 'Unf',
  1: 'BLQ',
  2: 'None',
  3: 'ALQ',
  4: 'Rec',
  5: 'LwQ',
  6: 'GLQ'},
 'BsmtQual': {0: 'Gd', 1: 'TA', 2: 'Ex', 3: 'None', 4: 'Fa'},
 'CentralAir': {0: 'Y', 1: 'N'},
 'Condition1': {0: 'Norm',
  1: 'Feedr',
  2: 'PosN',
  3: 'Artery',
  4: 'RRAe',
  5: 'RRNn',
  6: 'RRAn',
  7: 'PosA',
  8: 'RRNe'},
 'Condition2': {0: 'Norm',
  1: 'Artery',
  2: 'RRNn',
  3: 'Feedr',
  4: 'PosN',
  5: 'PosA',
  6: 'RRAn',
  7: 'RRAe'},
 'Electrical': {0: 'SBrkr', 1: 'FuseF', 2: 'FuseA', 3: 'FuseP', 4: 'Mix'},
 'ExterCond': {0: 'TA', 1: 'Gd', 2: 'Fa', 3: 'Po', 4: 'Ex'},
 'ExterQual': {0: 'Gd', 1: 'TA', 2: 'Ex', 3: 'Fa'},
 'Exte

In [60]:
for key,lis in object_value.items():
    if train[col].dtype=='O':
        for index,val in lis.items():
            train.loc[train[key]==val,key]=index
        
        print(key)
        train[key].astype('int64')



MSSubClass
MSZoning
Street
Alley
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
KitchenQual
Functional
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PavedDrive
PoolQC
Fence
MiscFeature
SaleType
SaleCondition


In [61]:
for x in train.columns:
    print(x,train[key].dtype)

MSSubClass int64
MSZoning int64
LotFrontage int64
LotArea int64
Street int64
Alley int64
LotShape int64
LandContour int64
Utilities int64
LotConfig int64
LandSlope int64
Neighborhood int64
Condition1 int64
Condition2 int64
BldgType int64
HouseStyle int64
OverallQual int64
OverallCond int64
YearBuilt int64
YearRemodAdd int64
RoofStyle int64
RoofMatl int64
Exterior1st int64
Exterior2nd int64
MasVnrType int64
MasVnrArea int64
ExterQual int64
ExterCond int64
Foundation int64
BsmtQual int64
BsmtCond int64
BsmtExposure int64
BsmtFinType1 int64
BsmtFinSF1 int64
BsmtFinType2 int64
BsmtFinSF2 int64
BsmtUnfSF int64
TotalBsmtSF int64
Heating int64
HeatingQC int64
CentralAir int64
Electrical int64
1stFlrSF int64
2ndFlrSF int64
LowQualFinSF int64
GrLivArea int64
BsmtFullBath int64
BsmtHalfBath int64
FullBath int64
HalfBath int64
BedroomAbvGr int64
KitchenAbvGr int64
KitchenQual int64
TotRmsAbvGrd int64
Functional int64
Fireplaces int64
FireplaceQu int64
GarageType int64
GarageYrBlt int64
GarageFini

In [75]:
x=train*train.T
t=train.astype('float64')
x=x.astype('float64')
p=np.linalg.matrix_rank(x)

/usr/local/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py:3140: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  return this.join(other, how=how, return_indexers=return_indexers)
/usr/local/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py:3140: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  return this.join(other, how=how, return_indexers=return_indexers)
/usr/local/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
/usr/local/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1647: RuntimeWarning: invalid value encountered in greater
  return count_nonzero(S > tol, axis=-1)


In [76]:
p


0

In [36]:

from sklearn import svm
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
outliers_fraction = 0.01
n_samples = 200

# define two outlier detection tools to be compared
outliers = {
    #"One-Class SVM": svm.OneClassSVM(nu=0.3, kernel="rbf", gamma=0.1)
    "Robust covariance": EllipticEnvelope(contamination=outliers_fraction),
    #"Isolation Forest": IsolationForest(max_samples=n_samples,contamination=outliers_fraction)
    #"Local Outlier Factor": LocalOutlierFactor(n_neighbors=35,contamination=outliers_fraction)
}

X=train
X['SalePrice']=target
outlier=set()

for clf_name,clf in outliers.items():
    if clf_name == "Local Outlier Factor":
        y_pred = clf.fit_predict(X)
        scores_pred = clf.negative_outlier_factor_
    else:
        clf.fit(X)
        scores_pred = clf.decision_function(X)
        y_pred = clf.predict(X)
       
        outlier=(np.where(y_pred==-1)[0])

In [37]:
train_new = train.drop(outlier)
train_new=train_new.drop('SalePrice',axis=1)
target_new = target.drop(outlier)


In [38]:
train_new.shape

(1445, 79)

In [39]:
from sklearn.ensemble import  BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error


clf={}
#clf['Bagging_KNR']= BaggingRegressor(KNeighborsRegressor())
#clf['Bagging_DT']= BaggingRegressor(DecisionTreeRegressor())
#clf['RandomForest']=RandomForestRegressor()
#clf['ExtraTrees']=ExtraTreesRegressor()
#clf['AdaBoost']=AdaBoostRegressor()
#clf['GradientBoost']=GradientBoostingRegressor()
clf['GradientBoost_NB']=GradientBoostingRegressor(loss='lad', alpha=0.95,
                                n_estimators=500, subsample=.5,
                                learning_rate=.1, max_leaf_nodes=10
                                )
clf['GradientBoost_NB_2']=GradientBoostingRegressor(loss='lad',
                                n_estimators=500, subsample=.5,
                                learning_rate=.05, max_leaf_nodes=10
                                )
clf['GradientBoost_NB_3']=GradientBoostingRegressor(loss='lad',
                                n_estimators=1000, subsample=.5,
                                learning_rate=.05, max_leaf_nodes=10
                                )
clf['GradientBoost_NB_4']=GradientBoostingRegressor(loss='lad',
                                n_estimators=1000, subsample=.5,
                                learning_rate=.01, max_leaf_nodes=10)
clf['GradientBoost_NB_ls']=GradientBoostingRegressor(
                                n_estimators=500, subsample=.5,
                                learning_rate=.1, max_leaf_nodes=10
                                )
clf['GradientBoost_NB_5']=GradientBoostingRegressor(loss='lad',
                                n_estimators=500, subsample=.5,
                                learning_rate=.05, max_leaf_nodes=5
                                )

#clf['GaussianNB']=GaussianNB()

for key,obj in clf.items():
    scores= np.sqrt(-cross_val_score(obj,train_new,target_new,scoring='neg_mean_squared_error'))

    print("{}: {:.6f}, {:.4f}".format(key,scores.mean(),scores.std()))
    obj.fit(train_new,target_new)

GradientBoost_NB: 27666.901921, 2854.8929
GradientBoost_NB_2: 28447.011727, 3222.1691
GradientBoost_NB_3: 27710.156522, 2978.7484
GradientBoost_NB_4: 29333.646839, 3640.9219


KeyboardInterrupt: 

In [ ]:
test=pd.read_csv('test.csv')

In [ ]:
test.head()

In [ ]:
Y=test.drop(["Id"],axis=1);Y.columns.size

In [ ]:
for col in Y.columns:
    if Y[Y[col].isnull()==True][col].size>0 :
        print(col,Y[Y[col].isnull()==True][col].size)

In [ ]:
for x in Y['MSSubClass'].values:
    Y.loc[Y['MSSubClass']==x,'MSSubClass']='sub'+str(x)

Y['MSSubClass'].astype('O')

In [ ]:
Y['GarageYrBlt']=Y['GarageYrBlt'].fillna(0)
Y['GarageYrBlt'].astype('int64')

In [ ]:
data['MSZoning'].value_counts()

In [ ]:
Y['MSZoning']=Y['MSZoning'].fillna('RL')
        

In [ ]:
Y['LotFrontage']=Y['LotFrontage'].fillna(Y['LotFrontage'].mean())

In [ ]:
Y['Alley']=Y['Alley'].fillna('None')

In [ ]:
data['Utilities'].value_counts()



In [ ]:
Y['Utilities']=Y['Utilities'].fillna('AllPub')




In [ ]:
data['Exterior1st'].value_counts()

In [ ]:
data['Exterior2nd'].value_counts()

In [ ]:
Y['Exterior1st']=Y['Exterior1st'].fillna('VinylSd')
Y['Exterior2nd']=Y['Exterior2nd'].fillna('VinylSd')



In [ ]:
Y[Y['MasVnrType'].isna()==True][['MasVnrType','MasVnrArea']]

In [ ]:
Y.loc[1150,'MasVnrArea']=0

In [ ]:
Y['MasVnrType']=Y['MasVnrType'].fillna('None')

In [ ]:
Y['MasVnrArea']=Y['MasVnrArea'].fillna(0)

In [ ]:
d=['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinSF1','BsmtFinType2','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath']

In [ ]:
Y[Y['BsmtCond'].isna()==True][d]

In [ ]:
data[(data['BsmtQual']=='TA')&(data['BsmtExposure']=='Av')][d]

In [ ]:
Y.loc[1064,'BsmtCond']='TA'
Y.loc[725,'BsmtCond']='TA'
Y.loc[580,'BsmtCond']='TA'

In [ ]:
for x in d:
    if Y[x].dtype=='O':
        Y[x]=Y[x].fillna('None')
    else:
        Y[x]=Y[x].fillna(0)



In [ ]:
for col in Y.columns:
    if Y[Y[col].isnull()==True][col].size>0 :
        print(col,Y[Y[col].isnull()==True][col].size)

In [ ]:
data['Functional'].value_counts()

In [ ]:
Y['KitchenQual']=Y['KitchenQual'].fillna('TA')

Y['Functional']=Y['Functional'].fillna('Typ')




In [ ]:
data['KitchenQual'].value_counts()

In [ ]:
Y[(Y['FireplaceQu'].isna()==True)&(Y['Fireplaces']!=0)][['FireplaceQu','Fireplaces']]

In [ ]:
Y['FireplaceQu']=Y['FireplaceQu'].fillna('None')



In [ ]:
d=['GarageType','GarageYrBlt','GarageFinish','GarageCars','GarageArea','GarageQual','GarageCond']

In [ ]:
Y[(Y['GarageYrBlt'].isna()==True)&(Y['GarageType'].isna()==False)][d]

In [ ]:
Y.loc[1116,'GarageType']='None'
Y.loc[666,'GarageType']='Detchd'



In [ ]:
Y[(Y['GarageType']=='Detchd')&(Y['GarageArea']==360)][d]

In [ ]:
data[(data['GarageType']=='Detchd')&(data['GarageArea']==360)][d]

In [ ]:
Y.loc[666,['GarageYrBlt','GarageFinish','GarageQual','GarageCond']]=[1965.0,'Unf','TA','TA']

In [ ]:
Y.loc[666,d]

In [ ]:
for x in d:
    if Y[x].dtype=='O':
        Y[x]=Y[x].fillna('None')
    else:
        Y[x]=Y[x].fillna(0)

In [ ]:
Y[(Y['PoolQC'].isna()==True)&(Y['PoolArea']!=0)][['PoolQC','PoolArea']]

In [ ]:
Y.loc[[960,1043,1139],'PoolQC']='Gd'

In [ ]:
Y['PoolQC']=Y['PoolQC'].fillna('None')
Y['Fence']=Y['Fence'].fillna('None')
Y['MiscFeature']=Y['MiscFeature'].fillna('None')

In [ ]:
Y['SaleType']=Y['SaleType'].fillna('WD')

In [ ]:
print(object_value['MSSubClass'],Y['MSSubClass'].unique())

In [ ]:
for key,lis in object_value.items():
    if Y[col].dtype=='O':
        for index,val in lis.items():
            if val in Y[key].unique():
                print(key,index,val)
                Y.loc[Y[key]==val,key]=index
        
        print(key)
        Y[key].astype('int64')

In [ ]:
for x in train.columns:
    print(x,train[key].dtype)

In [ ]:
for x in Y.columns:
    if 'None' in Y[x].unique():
        print(x,Y[x].unique())

In [ ]:
sub=clf['GradientBoost_NB_2'].predict(Y)

In [ ]:
submission=pd.DataFrame(sub,columns=['SalePrice'])

In [ ]:
submission['Id']=test['Id']

In [ ]:
submission.head()

In [ ]:
submission=submission[['Id','SalePrice']]

In [ ]:
submission.to_csv("submission.csv",index=False)